In [66]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply
import talib as ta
import importlib
import os
import platform
import psutil

In [68]:
# start = datetime.date(2021,1,1)
start = datetime.datetime.today() + relativedelta.relativedelta(years=-1)
end = datetime.date.today()
#https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=none&dir=info&file=domestic_info150818_01.html
# data = web.DataReader('7203.T', 'yahoo', start, end)
# data = web.DataReader('SPYD', 'yahoo', start, end)
# data = web.DataReader('1321.T', 'yahoo', start, end)
# data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
# data = web.DataReader('^N225', 'yahoo', start, end)
data = data.astype("double")
print(data)

initial = 1000000
fname = f"html/plot-"

             High    Low   Open  Close       Volume  Adj Close
Date                                                          
2021-09-13  370.0  364.0  367.0  365.0   33019794.0      365.0
2021-09-14  363.0  356.0  361.0  358.0   66934700.0      358.0
2021-09-15  367.0  361.0  364.0  362.0   54868371.0      362.0
2021-09-16  371.0  360.0  360.0  367.0   52270565.0      367.0
2021-09-17  367.0  362.0  366.0  362.0   43735573.0      362.0
...           ...    ...    ...    ...          ...        ...
2022-09-06  372.0  364.0  369.0  370.0   78379702.0      370.0
2022-09-07  380.0  371.0  372.0  374.0  110425487.0      374.0
2022-09-08  367.0  357.0  367.0  357.0  113510594.0      357.0
2022-09-09  357.0  352.0  354.0  353.0   99795902.0      353.0
2022-09-12  348.0  343.0  346.0  346.0   73706891.0      346.0

[245 rows x 6 columns]


**カスタムOptimization**

In [ ]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

import BacktestingStrategies.Optimization as opt
importlib.reload(opt)

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output2=bt.optimize(\
    upper_bound = range(50, 85, 5),
    lower_bound = range(10, 45, 5),
    rsi_window=range(10, 30, 2),\
    maximize=opt.optim_func,\
    method='grid',\
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot()

**MaxTries回数を設定**
**Constraintを設定**

In [ ]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

import BacktestingStrategies.Optimization as opt
importlib.reload(opt)

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output2=bt.optimize(\
    upper_bound = range(10, 85, 5),     #Constraintのお陰で変な値にならない
    lower_bound = range(10, 85, 5),     #Constraintのお陰で変な値にならない
    rsi_window=range(10, 30, 2),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: x.upper_bound > x.lower_bound,\
    max_tries = 100)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot()

**Heat Map**

In [ ]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

import BacktestingStrategies.Optimization as opt
importlib.reload(opt)

import seaborn as sns
import matplotlib.pyplot as plt

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output2, heatmap=bt.optimize(\
    upper_bound = range(10, 85, 5),     #Constraintのお陰で変な値にならない
    lower_bound = range(10, 85, 5),     #Constraintのお陰で変な値にならない
    rsi_window=14,\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: x.upper_bound > x.lower_bound,\
    max_tries = 100,
    return_heatmap=True)
print(output2)
display(output2._trades)
display(output2._strategy)

print(heatmap)
hml = heatmap.groupby(["upper_bound", "lower_bound"]).mean().unstack()
sns.heatmap(hml)
plt.show()
# bt.plot()

In [ ]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

import BacktestingStrategies.Optimization as opt
importlib.reload(opt)

import seaborn as sns
import matplotlib.pyplot as plt

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output2, heatmap=bt.optimize(\
    upper_bound = range(55, 85, 5),     #Constraintのお陰で変な値にならない
    lower_bound = range(10, 45, 5),     #Constraintのお陰で変な値にならない
    rsi_window= range(10, 45, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: x.upper_bound > x.lower_bound,\
    max_tries = 100,
    return_heatmap=True)
print(output2)
display(output2._trades)
display(output2._strategy)

print(heatmap)
plot_heatmaps(heatmap, agg = "mean")

**時系列データ resamply_apply**

In [ ]:
#週平均のRSIが上限下限を超えたとき、という条件を追加
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator_WithWeekly, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html") 


# output2=bt.optimize(\
#     upper_bound = range(50, 85, 5),
#     lower_bound = range(10, 45, 5),
#     rsi_window=range(10, 30, 2),\
#     maximize='Equity Final [$]',\
#     method='grid',\
#     max_tries = 100)
# print(output2)
# display(output2._trades)
# display(output2._strategy)
# bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

**空売りあり**

In [ ]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

import BacktestingStrategies.Optimization as opt
importlib.reload(opt)

import seaborn as sns
import matplotlib.pyplot as plt

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator_WithShortPosition, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + ".html") 

output2, heatmap=bt.optimize(\
    upper_bound = range(55, 85, 5),     #Constraintのお陰で変な値にならない
    lower_bound = range(10, 45, 5),     #Constraintのお陰で変な値にならない
    rsi_window= range(10, 45, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: x.upper_bound > x.lower_bound,\
    max_tries = 100,
    return_heatmap=True)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

**損切の設定**

In [ ]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

import BacktestingStrategies.Optimization as opt
importlib.reload(opt)

import seaborn as sns
import matplotlib.pyplot as plt

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator_WithStopLoss, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + ".html") 

output2, heatmap=bt.optimize(\
    upper_bound = range(55, 85, 5),     #Constraintのお陰で変な値にならない
    lower_bound = range(10, 45, 5),     #Constraintのお陰で変な値にならない
    rsi_window= range(10, 45, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: x.upper_bound > x.lower_bound,\
    max_tries = 100,
    return_heatmap=True)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

**単位買い（シグナルごと1株ずつ買う等、ドルコスト平均法とか）**

In [72]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

import BacktestingStrategies.Optimization as opt
importlib.reload(opt)

import seaborn as sns
import matplotlib.pyplot as plt

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator_BuySize, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + ".html") 

output2, heatmap=bt.optimize(\
    upper_bound = range(55, 85, 5),     #Constraintのお陰で変な値にならない
    lower_bound = range(10, 45, 5),     #Constraintのお陰で変な値にならない
    rsi_window= range(10, 45, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: x.upper_bound > x.lower_bound,\
    max_tries = 100,
    return_heatmap=True)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

Start                     2021-09-13 00:00:00
End                       2022-09-12 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                    7.346939
Equity Final [$]                 1000019.3566
Equity Peak [$]                  1000040.3566
Return [%]                           0.001936
Buy & Hold Return [%]               -5.205479
Return (Ann.) [%]                    0.001991
Volatility (Ann.) [%]                0.003257
Sharpe Ratio                         0.611215
Sortino Ratio                        0.957121
Calmar Ratio                         0.948117
Max. Drawdown [%]                     -0.0021
Avg. Drawdown [%]                   -0.000975
Max. Drawdown Duration        5 days 00:00:00
Avg. Drawdown Duration        5 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       5.801583
Worst Trade [%]                      5.801583
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,226,243,333.6434,353.0,19.3566,0.058016,2022-08-17,2022-09-09,23 days


<Strategy RsiOscillator_BuySize(upper_bound=55,lower_bound=40,rsi_window=20)>

Start                     2021-09-13 00:00:00
End                       2022-09-12 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   30.204082
Equity Final [$]                 1000078.0812
Equity Peak [$]                 1000105.09555
Return [%]                           0.007808
Buy & Hold Return [%]               -5.205479
Return (Ann.) [%]                    0.008031
Volatility (Ann.) [%]                0.007503
Sharpe Ratio                         1.070404
Sortino Ratio                        1.735294
Calmar Ratio                         1.732132
Max. Drawdown [%]                   -0.004637
Avg. Drawdown [%]                   -0.001582
Max. Drawdown Duration       41 days 00:00:00
Avg. Drawdown Duration       14 days 00:00:00
# Trades                                   11
Win Rate [%]                        27.272727
Best Trade [%]                       15.70025
Worst Trade [%]                     -7.561396
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,127,128,387.91070,384.0,-3.91070,-0.010081,2022-03-23,2022-03-24,1 days
1,1,128,129,385.90080,383.0,-2.90080,-0.007517,2022-03-24,2022-03-25,1 days
2,1,129,131,384.89585,381.0,-3.89585,-0.010122,2022-03-25,2022-03-29,4 days
3,1,131,132,382.88595,381.0,-1.88595,-0.004926,2022-03-29,2022-03-30,1 days
4,1,132,159,382.88595,443.0,60.11405,0.157002,2022-03-30,2022-05-12,43 days
5,1,178,179,366.80675,365.0,-1.80675,-0.004926,2022-06-08,2022-06-09,1 days
6,1,179,183,366.80675,419.0,52.19325,0.142291,2022-06-09,2022-06-15,6 days
7,1,209,224,367.81170,340.0,-27.81170,-0.075614,2022-07-22,2022-08-15,24 days
8,1,224,225,341.68300,341.0,-0.68300,-0.001999,2022-08-15,2022-08-16,1 days
9,1,225,226,342.68795,332.0,-10.68795,-0.031189,2022-08-16,2022-08-17,1 days


<Strategy RsiOscillator_BuySize(upper_bound=65,lower_bound=30,rsi_window=10)>

Row(id='57224', ...)

**●●の指標が××から何日間経過したか**

In [74]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

import BacktestingStrategies.Optimization as opt
importlib.reload(opt)

import seaborn as sns
import matplotlib.pyplot as plt

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator_BarsSince, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + ".html") 

output2, heatmap=bt.optimize(\
    upper_bound = range(55, 85, 5),     #Constraintのお陰で変な値にならない
    lower_bound = range(10, 45, 5),     #Constraintのお陰で変な値にならない
    rsi_window= range(10, 45, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: x.upper_bound > x.lower_bound,\
    max_tries = 100,
    return_heatmap=True)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

Start                     2021-09-13 00:00:00
End                       2022-09-12 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                    7.346939
Equity Final [$]                 1000019.3566
Equity Peak [$]                  1000040.3566
Return [%]                           0.001936
Buy & Hold Return [%]               -5.205479
Return (Ann.) [%]                    0.001991
Volatility (Ann.) [%]                0.003257
Sharpe Ratio                         0.611215
Sortino Ratio                        0.957121
Calmar Ratio                         0.948117
Max. Drawdown [%]                     -0.0021
Avg. Drawdown [%]                   -0.000975
Max. Drawdown Duration        5 days 00:00:00
Avg. Drawdown Duration        5 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       5.801583
Worst Trade [%]                      5.801583
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,226,243,333.6434,353.0,19.3566,0.058016,2022-08-17,2022-09-09,23 days


<Strategy RsiOscillator_BarsSince(upper_bound=65,lower_bound=25,rsi_window=10)>

Start                     2021-09-13 00:00:00
End                       2022-09-12 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                    7.346939
Equity Final [$]                 1000019.3566
Equity Peak [$]                  1000040.3566
Return [%]                           0.001936
Buy & Hold Return [%]               -5.205479
Return (Ann.) [%]                    0.001991
Volatility (Ann.) [%]                0.003257
Sharpe Ratio                         0.611215
Sortino Ratio                        0.957121
Calmar Ratio                         0.948117
Max. Drawdown [%]                     -0.0021
Avg. Drawdown [%]                   -0.000975
Max. Drawdown Duration        5 days 00:00:00
Avg. Drawdown Duration        5 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       5.801583
Worst Trade [%]                      5.801583
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,226,243,333.6434,353.0,19.3566,0.058016,2022-08-17,2022-09-09,23 days


<Strategy RsiOscillator_BarsSince(upper_bound=55,lower_bound=30,rsi_window=15)>

Row(id='62040', ...)